Reference: 
- https://d2l.ai/chapter_recommender-systems/fm.html
- https://github.com/khanhnamle1994/MetaRec/blob/master/Matrix-Factorization-Experiments/Factorization-Machines/FM.py
- https://github.com/qian135/ctr_model_zoo/blob/master/common.py

Original paper:
https://www.csie.ntu.edu.tw/~b97053/paper/Rendle2010FM.pdf

Factorization Machines
- a generic algorithm for classification, regression and ranking
- a generalization of linear regression and matrix factorization model
- similar to SVM with a polynomial kernel
- models n-way (typically 2-way) interaction between features (embeddings)
- compute 2-way feature interaction as the dot product of the latent features
- author proposes a neat trick to reduce polynomial complexity to linear time

the model (2-way interaction): 

$\hat{y}(x) = \mathbf{w}_0 + \sum_{i=1}^d \mathbf{w}_i x_i + \sum_{i=1}^d\sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j$

- $w_0$ - global bias
- $w_i$ - weight of i-th variable
- $V, v_i, v_j$ - feature embeddings
- $\langle\mathbf{v}_i, \mathbf{v}_j\rangle$ - dot product of embeddings -> weight of the interaction

As we can see, the first part of the equation is just a linear regression

derivation of pairwise interaction

\begin{split}\begin{aligned}
&\sum_{i=1}^d \sum_{j=i+1}^d \langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j \\
 &= \frac{1}{2} \sum_{i=1}^d \sum_{j=1}^d\langle\mathbf{v}_i, \mathbf{v}_j\rangle x_i x_j - \frac{1}{2}\sum_{i=1}^d \langle\mathbf{v}_i, \mathbf{v}_i\rangle x_i x_i \\
 &= \frac{1}{2} \big (\sum_{i=1}^d \sum_{j=1}^d \sum_{l=1}^k\mathbf{v}_{i, l} \mathbf{v}_{j, l} x_i x_j - \sum_{i=1}^d \sum_{l=1}^k \mathbf{v}_{i, l} \mathbf{v}_{i, l} x_i x_i \big)\\
 &=  \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i) (\sum_{j=1}^d \mathbf{v}_{j, l}x_j) - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2 \big ) \\
 &= \frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i)^2 - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2)
 \end{aligned}\end{split}

 - $k$ - embedding dimension
 - $d$ - number of features

An illustration of the shape & format of the data, where each feature will be embedded into a latent space

<img src="data_illustration.png" style="width:600px;height:300px;">


In [99]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils import data
from torch.utils.data import DataLoader
from sklearn.metrics import mean_squared_error, mean_absolute_error

Factorization Machines can be very easily implemented in Pytorch with just a few lines of code!

In [65]:
import torch
import torch.nn as nn
class FM(nn.Module):
    def __init__(self, n_features, embed_dim):
        super().__init__()
        self.feature_embedding = nn.Embedding(n_features, embed_dim) # n_dim feature embeddings for 2-way interaction
        self.fc = nn.Embedding(n_features, 1) # 1-d embedding equivalent to the weights for each feature in linear regression
        self.bias = nn.Parameter(torch.zeros((1,))) # global bias term
        
        
    def forward(self, x):
        first_order = self.fc(x).sum(dim=1) + self.bias
        second_order = self.factorization_machine(self.feature_embedding(x))
        res = first_order + second_order
        return res
    
    
    def factorization_machine(self, x_embed):
        """compute the 2-way interaction term
        """
        sq_sum = x_embed.sum(dim=1)**2 # (x_size, n_dim)
        sum_sq = (x_embed**2).sum(dim=1) # (x_size, n_dim)
        return 0.5 * (sq_sum - sum_sq).sum(dim=1, keepdim=True) # (x_size, 1)
    
    def predict(self, x):
        self.eval()
        with torch.no_grad():
            return self.forward(x)
    
        

The `factorization_machine()` function implements the equation:

$\frac{1}{2} \sum_{l=1}^k \big ((\sum_{i=1}^d \mathbf{v}_{i, l} x_i)^2 - \sum_{i=1}^d \mathbf{v}_{i, l}^2 x_i^2)$

There are two parts in the equation:

$(\mathbf{v}_{i, l} x_i)^2$     and     $(\mathbf{v}_{i, l}^2 x_i^2)$

Since each $x_i$ corresponds to one $v_i$, we can just treat $\mathbf{v}_{i, l} x_i$ as one latent embedding vector, which will be estimated during training, instead of estimating $V$ separately.

In [64]:
# train loop
def train(model, dataloader, epochs=20, lr=0.001):
    device = (
        torch.device("cuda:0") if torch.cuda.is_available(
        ) else torch.device("cpu")
    )
    model.to(device)
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    training_history = []
    for epoch in range(epochs):
        epoch_loss = 0
        for x, y in dataloader:
            y_pred = model.forward(x)
            loss = criterion(y_pred, y)
            epoch_loss += loss
            model.zero_grad()
            loss.backward()
            optimizer.step()
        epoch_loss /= len(dataloader)
        training_history.append(epoch_loss)
        if epoch%10 == 0:
            print(f"Epoch {epoch}: {epoch_loss:.4f}")
    return model, training_history


# Data Preparation
- X is an array of feature indices (n_samples, n_attributes), where each feature index will be mapped to a latent factor
  - [user, item, user_features, item_features]
- y is just a (n_sample, 1) array of the ground truth

In [6]:
import sys
sys.path.append('..')
import utils

In [7]:
movie, rating = utils.load_movielens()

Encode movie features

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer
enc = MultiLabelBinarizer()
multi_hot = enc.fit_transform([set([str(j) for j in i]) for i in movie['features']])

# convert multi_hot_encoding to indices
# e.g. [0, 2, 5] -> 1st & 2nd features are absent, 3rd feature is present
features_enc = multi_hot + np.arange(multi_hot.shape[1])*2
features_enc

array([[ 0,  2,  4, ..., 58, 60, 62],
       [ 0,  2,  4, ..., 58, 60, 62],
       [ 0,  2,  4, ..., 58, 60, 62],
       ...,
       [ 0,  2,  4, ..., 58, 60, 62],
       [ 0,  2,  4, ..., 58, 60, 62],
       [ 0,  2,  4, ..., 58, 60, 62]])

Combine userId, movieId and features to get our `x`
- [userId, movieId, feature-1, feature-n]

In [16]:
n_user = rating['userId'].nunique()
n_movies = movie['movieId'].nunique()
n_movie_features = features_enc.shape[-1]*2

In [22]:
x = rating[['userId', 'movieId']].to_numpy()
features = features_enc[rating['movieId']]

# concat userid, movieid, and features to get x
x = np.hstack((x, features))
x

array([[   0,   30,    0, ...,   58,   60,   62],
       [   0,  833,    0, ...,   58,   60,   62],
       [   0,  859,    0, ...,   59,   60,   62],
       ...,
       [ 670, 4603,    0, ...,   59,   60,   62],
       [ 670, 4616,    0, ...,   58,   60,   62],
       [ 670, 4703,    0, ...,   58,   60,   62]])

In [23]:
# calculate offset, avoid duplicated feature indices
x[:, 1] += n_user
x[:, 2:] += n_user+n_movies
x

array([[   0,  701, 9796, ..., 9854, 9856, 9858],
       [   0, 1504, 9796, ..., 9854, 9856, 9858],
       [   0, 1530, 9796, ..., 9855, 9856, 9858],
       ...,
       [ 670, 5274, 9796, ..., 9855, 9856, 9858],
       [ 670, 5287, 9796, ..., 9854, 9856, 9858],
       [ 670, 5374, 9796, ..., 9854, 9856, 9858]])

In [39]:
y = rating['rating'].to_numpy().reshape(-1, 1)

Create train & test set
- train: first n-1 ratings per user (n is the number of ratings of the user)
- test: last/most recent rating per user

In [35]:
test_ix = [i for i,v in enumerate(x[:-1, 0]) if v != x[i+1,0]] + [len(x)-1]
train_ix = [i for i in range(len(x)) if i not in test_ix]

In [40]:
x_train, x_test = x[train_ix], x[test_ix]
y_train, y_test = y[train_ix], y[test_ix]

In [73]:
dataset = data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train).float())
train_dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Train Model

In [84]:
n_features = n_user + n_movies + n_movie_features*2
embed_dim = 40

model = FM(n_features=n_features, embed_dim=embed_dim)

In [85]:
model, history = train(model, train_dataloader, epochs=200, lr=0.001)

Epoch 0: 2158.6504
Epoch 10: 55.0315
Epoch 20: 19.3444
Epoch 30: 9.8613
Epoch 40: 5.8314
Epoch 50: 3.8517
Epoch 60: 2.7472
Epoch 70: 2.2466
Epoch 80: 1.8223
Epoch 90: 1.5533
Epoch 100: 1.4293
Epoch 110: 1.2750
Epoch 120: 1.2069
Epoch 130: 1.1149
Epoch 140: 1.0458
Epoch 150: 0.9703
Epoch 160: 0.8981
Epoch 170: 0.8504
Epoch 180: 0.8140
Epoch 190: 0.7846
